# 树莓派垃圾检测小车

使用PaddleDetection实现自定义数据集标注及模型训练并导出侧端部署.nb模型
**（前置项目版本）**

工具：精灵标注、PaddleDetection、PaddleLite、

<font size = 3 color = blue>[PaddleDetection官方文档](https://github.com/PaddlePaddle/PaddleDetection)</font>

<font size = 3 color = blue>[Paddle-Lite官方文档](https://github.com/PaddlePaddle/Paddle-Lite)</font>

# 一、项目背景

随着人民对物质生活要求的日益提高，生活产生的垃圾日益增多，现存的区域卫生管理方案已经不能满足大型商场、校园、景区、社区对于环境卫生方面的需求。传统的环境卫生维护方案人力成本高，效率低，本项目提出了一种以机械代替人工的智能化解决方案。

树莓派垃圾检测车采用PaddleDetection训练垃圾检测模型，然后使用Paddle-Lite的OPT工具实现模型的优化部署。最后配合驱动可对园区内垃圾实现精确检测，并利用自身携带机械臂拾取垃圾。
<center>

![](https://ai-studio-static-online.cdn.bcebos.com/2851bcf602e64e629854dc75e39e63cf0c309825945a42da8ce771e8448a404b)
</center>

# 二、数据集简介

<font size = 3 color = blue>我们选用了9类生活垃圾共650张图片，并使用精灵标注进行打标。相似的标注工具还有Labelme以及百度BML全功能AI开发平台！</font>

## 1.数据打标

#### 1、新建目标检测任务标注

<center>
<img src=https://ai-studio-static-online.cdn.bcebos.com/ec1b428f918c4be1be89cb4f9f78be963147ca02880f495e8715acd60754e9c7 width="600" >
</center>

#### 2、开始打框，打框前记得设置好label

<center>
<img src=https://ai-studio-static-online.cdn.bcebos.com/e0b26028976548ba80f05e1325b32a3e2c79c3902d8648c2a9a476616c3e5a11 width="600" >
</center>

#### 3、导出选择pascal-voc

<center>
<img src=https://ai-studio-static-online.cdn.bcebos.com/d7de4a4a610a4d009a19cd8c2c36830ccf4d7202bff14e1a95c4977563ba8a06 width="600" >
</center>


## 2.数据加载和预处理
<font size = 3 color = blue>此处我组织的数据集放在PaddleDetection/dataset/roadsign_voc目录下，可在下文解压数据集后自行查看。</font>

#### 1、整理成VOC格式的数据集： 创建三个文件夹：annotations、images、ImageSets。

![](https://ai-studio-static-online.cdn.bcebos.com/ac3fd601a42744c2b478a7005b1dc36b35acd25362304111818350929741d9dc)

#### 2、将标注生成的XML文件存入annotations，图片存入images，训练集、测试集、验证集的划分情况存入ImageSets。 在ImageSets下创建一个Main文件夹，并且在Mian文件夹下建立label_list.txt，里面存入标注的标签。 此label_list.txt文件复制一份与annotations、images、ImageSets同级位置放置。 其内容如下：

![](https://ai-studio-static-online.cdn.bcebos.com/daaa0bef743d46299f8fc9f48313f370731a6bd626ff4f8ea9be19a6888fd384)

#### 3、之后运行下列代码，会在Main文件夹下生成训练集、验证集、测试集的txt文件。

```
import os
import random

trainval_percent = 0.95
train_percent = 0.9
xmlfilepath = 'F:/Data/LaJiJianCheVoc/annotations' # 修改为自己的路径
txtsavepath = 'F:/Data/LaJiJianCheVoc/ImageSets/Main' # 修改为自己的路径
total_xml = os.listdir(xmlfilepath)

num = len(total_xml)
list = range(num)  # 650
tv = int(num * trainval_percent)  # 0.95 * 650 = 617.5
tr = int(num * train_percent)  # 0.9 * 650 = 585
trainval = random.sample(list, tv)
train = random.sample(list, tr)

ftrainval = open('F:/Data/LaJiJianCheVoc/ImageSets/Main/trainval.txt', 'w')
ftest = open('F:/Data/LaJiJianCheVoc/ImageSets/Main/test.txt', 'w')
ftrain = open('F:/Data/LaJiJianCheVoc/ImageSets/Main/train.txt', 'w')
fval = open('F:/Data/LaJiJianCheVoc/ImageSets/Main/val.txt', 'w')

for i in list:
    name = total_xml[i][:-4] + '\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close
ftrain.close
ftest.close
fval.close
```
#### 4、再运行以下可根据Main文件夹中划分好的数据集进行位置索引，生成具有图像路径和其对应的XML文件路径的txt文件。

```
'''
在Main文件夹中划分好的数据集进行位置索引，生成含有图像及对应的XML文件的地址信息的文件。
'''
import os
import random
import re

devkit_dir = './'
output_dir = './'

def get_dir(devkit_dir, type):
    return os.path.join(devkit_dir, type)

def walk_dir(devkit_dir):
    filelist_dir = get_dir(devkit_dir, 'ImageSets/Main') # 不需要更换路径
    annotation_dir = get_dir(devkit_dir, 'annotations')
    img_dir = get_dir(devkit_dir, 'images')
    trainval_list = []
    train_list = []
    val_list = []
    test_list = []

    added = set()  # 创建无序集合

    for _, _, files in os.walk(filelist_dir):
        for fname in files:
            img_ann_list = []
            if re.match('trainval.txt', fname):
                img_ann_list = trainval_list
            elif re.match('train.txt', fname):
                img_ann_list = train_list
            elif re.match('val.txt', fname):
                img_ann_list = val_list
            elif re.match('test.txt', fname):
                img_ann_list = test_list
            else:
                continue
            fpath = os.path.join(filelist_dir, fname)
            for line in open(fpath):
                name_prefix = line.strip().split()[0]
                # print(name_prefix)

                added.add(name_prefix)
                ann_path = annotation_dir + '/' + name_prefix + '.xml'
                # print(ann_path)
                img_path = img_dir + '/' + name_prefix + '.jpg'
                assert os.path.isfile(
                    ann_path), 'file %s not found.' % ann_path
                assert os.path.isfile(
                    img_path), 'file %s not found.' % img_path
                img_ann_list.append((img_path, ann_path))

            print(img_ann_list[:4])
    return trainval_list, train_list, test_list, val_list

def prepare_filelist(devkit_dir, output_dir):
    trainval_list = []
    train_list = []
    test_list = []
    val_list = []

    trainval, train, test, val = walk_dir(devkit_dir)

    trainval_list.extend(trainval)
    train_list.extend(train)
    test_list.extend(test)
    val_list.extend(val)

    with open(os.path.join(output_dir, 'trainval.txt'), 'w') as ftrainval:
        for item in trainval_list:
            ftrainval.write(item[0] + ' ' + item[1] + '\n')
    with open(os.path.join(output_dir, 'train.txt'), 'w') as ftrain:
        for item in train_list:
            ftrain.write(item[0] + ' ' + item[1] + '\n')
    with open(os.path.join(output_dir, 'test.txt'), 'w') as ftest:
        for item in test_list:
            ftest.write(item[0] + ' ' + item[1] + '\n')
    with open(os.path.join(output_dir, 'val.txt'), 'w') as fval:
        for item in val_list:
            fval.write(item[0] + ' ' + item[1] + '\n')


if __name__ == '__main__':
    prepare_filelist(devkit_dir, output_dir)
```

#### 5、最后整个文件目录如下 <font size = 3 color = blue>(注意.py文件需要放在文件夹根目录)</font>

![](https://ai-studio-static-online.cdn.bcebos.com/6f6d1de44c3d46058fd81474b55e01a3c33c10ee3cf045c3b8c609545925978d)

#### 6、最后只需要将annotations、images文件夹，以及label_list.txt、test.txt、train.txt、val.txt、trainval.txt文件上传至PaddleDetection/dataset/roadsign_voc目录下。

## 3.修改网络配置

<font size = 3 color = blue> 此处我们使用的是ssd_mobilenet_v1_300_120e_voc.yml网络的配置文件，相关注释我已经标注如下：</font>

<font size = 3 color = blue>在PaddleDetection 2.0后续版本，采用了模块解耦设计，用户可以组合配置模块实现检测器，并可自由修改覆盖各模块配置，如下图所示</font>
</font>
![](https://ai-studio-static-online.cdn.bcebos.com/c6e0296caad845b785be7680a192719aa990fb645eb747a4aed021f2171273c7)

<font size = 3 color = blue> 由于原网络配置和我们需要的配置有出入，具体体现在我们此处做的9类垃圾检测，而原配置不是。我们需要做几处修改：1、修改num_class为10(9label_class)+1(background)。2、修改voc.py文件中的label。</font>

<font size = 3 color = blue>1、修改PaddleDetection/configs/datasets/roadsign_voc.yml</font>

![](https://ai-studio-static-online.cdn.bcebos.com/1763fa5325b04744871137871b179f11613a7cf49fe74c1b848297d37064f096)

<font size = 3 color = blue>2、修改PaddleDetection/ppdet/data/source/voc.py(值得注意的是此处的if语句在前面num_class为10的时候一定要加上)</font>

![](https://ai-studio-static-online.cdn.bcebos.com/a4f2e36dc3034d239c00bba6fcb18bae0e607483d1da4f9e978992e799982882)


# 三、代码实操

In [1]:
import paddle
print(paddle.__version__)

2.1.0


In [ ]:
# 下载paddleDetection
!pip install paddledet==2.1.0 -i https://mirror.baidu.com/pypi/simple

In [ ]:
# 克隆paddledetetection库（github网速感人已换gitee）
!git clone https://gitee.com/PaddlePaddle/PaddleDetection.git

In [4]:
# 设置工作目录
%cd PaddleDetection/
%env PYTHONPATH=/home/aistudio/PaddleDetection

In [ ]:
# 验证是否安装成功
!python tools/infer.py \
        -c configs/ppyolo/ppyolo_r50vd_dcn_1x_coco.yml \
        -o use_gpu=true \
        weights=https://paddledet.bj.bcebos.com/models/ppyolo_r50vd_dcn_1x_coco.pdparams \
        --infer_img=demo/000000014439.jpg

<font size = 4 color=blue>若验证安装成功，在PaddleDetection/output目录下会出现下图检测结果图片</font>

<center>
  
![](https://ai-studio-static-online.cdn.bcebos.com/6f79ea3f41f54f1c97c31abfaa9f1cfb74812445350b463ea361428e90708a56)
</center>

<font size = 4 color=blue>若验证安装失败，请对照官方文档解决</font>

In [7]:
%cd ~

# 解压数据集，若前面已经上传就不要再运行
!unzip -oq data/data95917/LaJiJianCheVoc.zip -d PaddleDetection/dataset/roadsign_voc/

%cd PaddleDetection/

## 开始训练

<font size = 4 color=blue>训练开始之前请保证上述文件均已按照要求修改完成，此处我将已经修改好的文件放在work/Revised_document目录下，也在相应路径下作出替换，读者可自行查看。</font>

In [8]:
!python -u tools/train.py \
        -c configs/ssd/ssd_mobilenet_v1_300_120e_voc.yml \
        --eval \
        --use_vdl=True \
        --vdl_log_dir=output/vdl_dir/scalar

<font size = 4 color=blue>出现ppdet.engine INFO: Eval iter: 0一直为0的情况是因为eval里面的数据量少于100，大家可以自行扩充。</font>

#### 推理验证

In [14]:
!python tools/infer.py -c configs/ssd/ssd_mobilenet_v1_300_120e_voc.yml --infer_img=img_950.jpg

<font size = 4 color = blue>随手拍了一张照片，预测结果保存在output/test.jpg，效果来看不错。预测正确，label为易拉罐。</font>

<center>
  
![](https://ai-studio-static-online.cdn.bcebos.com/05b57c9d118f4d0f81978a4fb74d9bd2d6232e21dd04406f9dfb2c074792e0cb)
</center>

## 转化为预测模型

In [ ]:
!python -u tools/export_model.py -c configs/ssd/ssd_mobilenet_v1_300_120e_voc.yml --output_dir=./inference_model_final

#### 若要设置导出模型的输入大小，需要设置TestReader中的 `image_shape` 可以修改保存模型中的输入图片大小

```
!python -u tools/export_model.py \
		-c configs/ssd/ssd_mobilenet_v1_300_120e_voc.yml \
		--output_dir=./inference_model_final \
		-o TestReader.inputs_def.image_shape=[3,224,224] # 修改此处
```

# 四、Paddle-Lite侧端部署

<font size = 3 color = blue>Lite部署流程，在完成了模型训练流程之后，我们得到了paddle的原生推理模型，然后我们需要使用Opt工具生成我们的Lite优化模型，优化后的模型更轻量，模型运行速度更快。</font>

<center>
<img src=https://ai-studio-static-online.cdn.bcebos.com/d932932ec1254d6392292511a1c4065d253e9e5e28f84095b75503db28fa597a width="600" >
</center>

<font size = 3 color = blue>安装paddle_lite_opt工具有两种方法，下面将演示。</font>

## [建议] pip安装paddlelite并进行转换

In [16]:
!pip install paddlelite

In [17]:
# 转换工作目录
%cd ~

#### 使用OPT工具生成侧端部署模型

<font size = 4 color = blue>opt工具使用教程</font>

<font size = 4 color = blue>[教程链接🔗](https://paddle-lite.readthedocs.io/zh/latest/user_guides/opt/opt_bin.html)</font>

<center>
<img src=https://ai-studio-static-online.cdn.bcebos.com/a104064f60b140d988b03296bcb65a8d9ebc1d60064e467181928b2af73dc67f width="600" >
</center>

In [18]:
# 运行下列代码会在~根目录生成LaJiJianCheModle.nb文件
!paddle_lite_opt --model_file=PaddleDetection/inference_model_final/ssd_mobilenet_v1_300_120e_voc/model.pdmodel \
                 --param_file=PaddleDetection/inference_model_final/ssd_mobilenet_v1_300_120e_voc/model.pdiparams \
                 --optimize_out_type=naive_buffer \
                 --optimize_out=./LaJiJianCheModle

## 源码编译Paddle-Lite生成opt工具

#### 模型优化需要Paddle-Lite的opt可执行文件，可以通过编译Paddle-Lite源码获得，编译步骤如下：

In [ ]:
!git clone https://gitee.com/paddlepaddle/paddle-lite.git
%cd paddle-lite/

In [ ]:
# 检查分支
!git checkout develop

In [ ]:
# 启动编译 (时间较长，我的网络太拉了，读者可自行尝试）
!./lite/tools/build.sh build_optimize_tool

#### 编译完成后，`opt` 文件位于`build.opt/lite/api/`下，可通过如下方式查看`opt`的运行选项和使用方式；

In [1]:
cd build.opt/lite/api/
./opt
# 后面转化模型使用方法如1、

# 五、总结与升华 

这个项目是部署的前置项目，整个流程相对简单，读者们可以轻易复现，遇到问题也欢迎积极到评论区留言 : )
 
后期会完成此项目的部署工作，完成之后会在此项目更新链接。

# 关于作者

> 湖北文理学院 汽车与交通工程学院 车辆新能源 2019级本科生 马飞

> 主要方向：CV、RL、SLAM、AI安全

> 爱好：游泳、摸鱼、比赛、平常也会写写专利论文啥的

> 我的主页：[鱼不辞水](https://aistudio.baidu.com/aistudio/usercenter)  欢迎来互关啊！！！
